In [3]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics import v_measure_score, homogeneity_score, completeness_score
from tribus import marker_expression
from minisom import MiniSom

from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import time

# Read in datasets

In [8]:
data_type = "CODEX"
Q = 0.999
Logic = False

In [9]:
if data_type == "CODEX": 

    # read in CODEX donor 004 cl dataset
    sample_data = pd.read_csv('C:\\Users\\Public\\Farkkila_lab_datasets\\Tribus\\Test_case_data\\STELLAR\\input_data\\STELLAR_data_donor_004_upperlevel_CL.csv',low_memory=False)
    print(np.shape(sample_data))

    #  perform always outlier truncation, set the maximum to the 99。9 percentile
    cols = ['MUC2', 'SOX9', 'MUC1', 'CD31', 'Synapto', 'CD49f', 'CD15',
        'CHGA', 'CDX2', 'ITLN1', 'CD4', 'CD127', 'Vimentin', 'HLADR',
        'CD8', 'CD11c', 'CD44', 'CD16', 'BCL2', 'CD3', 'CD123', 'CD38',
        'CD90', 'aSMA', 'CD21', 'NKG2D', 'CD66', 'CD57', 'CD206', 'CD68',
        'CD34', 'aDef5', 'CD7', 'CD36', 'CD138', 'CD45RO', 'Cytokeratin',
        'CK7', 'CD117', 'CD19', 'Podoplanin', 'CD45', 'CD56', 'CD69',
        'Ki67', 'CD49a', 'CD163', 'CD161']

    df = pd.ExcelFile("C:\\Users\\Public\\Farkkila_lab_datasets\\Tribus\\Test_case_data\\STELLAR\\stellar_logic_gate_cl_benchmarking.xlsx")
    logic = pd.read_excel(df, df.sheet_names, index_col=0)
    markers = list(logic["Global"].index)

    celltype1 = "cell_type_A"
    celltype2 = "cell_type_upperlevel"

elif data_type == "TMA": 

    # read input files
    # no outlier filtering
    sample_data = pd.read_csv("C:\\Users\\Public\\Farkkila_lab_datasets\\Tribus\\Test_case_data\\TMA_works\\TMA_all_data_labeled.csv")
    print(np.shape(sample_data))

    #  perform always outlier truncation, set the maximum to the 99。9 percentile
    cols = ['CD11c', 'CD1c', 'CD4', 'CD3d', 'CD20', 'CD163',
        'CD8a', 'cCasp3', 'pSTAT1', 'Ki67', 'PDL1', 'IBA1', 'FOXP3', 'PD1',
        'Ecadherin', 'vimentin', 'CD31', 'P21', 'CK7', 'CD45']
    
    celltype1 = "CellType"
    celltype2 = "merged_labels"

    df = pd.ExcelFile("C:\\Users\\Public\\Farkkila_lab_datasets\\Tribus\\Test_case_data\\TMA_works\\cell_type_descriptions.xlsx")
    logic = pd.read_excel(df, df.sheet_names, index_col=0)
    markers = list(logic["Global"].index)
    
elif data_type == "CyTOF": 

    # read input files
    # no outlier filtering
    sample_data = pd.read_csv("C:\\Users\\Public\\Farkkila_lab_datasets\\Tribus\\Test_case_data\\CyTOF_TNBC\\input_data\\TNBC_Data_origin.csv")
    print(np.shape(sample_data))

    #  perform always outlier truncation, set the maximum to the 99。9 percentile
    cols = ['Vimentin', 'SMA','FoxP3', 'Lag3', 'CD4', 
            'CD16', 'CD56', 'PD1', 'CD31','PD-L1', 
            'EGFR', 'Ki67', 'CD209', 'CD11c', 'CD138', 
            'CD163','CD68', 'CD8', 'CD3', 'IDO', 
            'Keratin17', 'CD63','CD45RO', 'CD20', 'p53', 
            'Beta catenin', 'HLA-DR', 'CD11b', 'CD45',
            'Pan-Keratin', 'MPO','Keratin6']
    
    celltype1 = "DetailedGroup"
    celltype2 = "Group"

    df = pd.ExcelFile("C:\\Users\\Public\\Farkkila_lab_datasets\\Tribus\\Test_case_data\\CyTOF_TNBC\\logic_gate.xlsx")
    logic = pd.read_excel(df, df.sheet_names, index_col=0)
    markers = list(logic["Global"].index)

Q = sample_data[cols].quantile(Q)
sample_data = sample_data[~((sample_data[cols] > Q)).any(axis=1)]
print(np.shape(sample_data))
sample_data[cols].describe()

labels_true_1 = sample_data[celltype1]
print(np.unique(labels_true_1))
labels_true_2 = sample_data[celltype2]
print(np.unique(labels_true_2))

grid_size = int(np.sqrt(np.sqrt(len(sample_data)) * 5))
print(data_type)
print(grid_size)

if logic == "FALSE":    
    marker_data = np.arcsinh(sample_data[cols]).to_numpy()
else: 
    marker_data = np.arcsinh(sample_data[markers]).to_numpy()


(124623, 36)
(122352, 36)
['Apoptotic' 'B-cells' 'CD11c+APC' 'CD11c+CD163+IBA1+Macrophages'
 'CD11c+IBA1+Macrophages' 'CD163+IBA1+Macrophages' 'CD163+Macrophages'
 'CD4+effectorT-cells' 'CD8+T-cells' 'EMT' 'Endothelia' 'Epithelial'
 'FOXP3+CD4+Tregs' 'Functional_epithelial' 'Functional_stroma' 'High-P21'
 'High-Vimentin' 'High-proliferative_Stroma' 'IBA1+Macrophages'
 'Low-vimentin' 'Low_eccentricity' 'Mesenchymal'
 'Non-proliferative_Stroma' 'Other' 'Proliferating EMT'
 'Proliferating epithelial' 'Proliferative_Stroma']
['B-cells' 'CD4 T cells' 'CD8 T cells' 'Myeloid' 'Stromal' 'T Reg cells'
 'Tumor' 'other_Global']
TMA
41


In [10]:
# Initialization and training
som_shape = (grid_size, grid_size)
som = MiniSom(som_shape[0], som_shape[1], marker_data.shape[1], sigma=.5, learning_rate=.5,
              neighborhood_function='gaussian')

som.train_batch(marker_data, 500, verbose=True)

# each neuron represents a cluster
winner_coordinates = np.array([som.winner(x) for x in marker_data]).T
# with np.ravel_multi_index we convert the bidimensional
# coordinates to a monodimensional index
cluster_index = np.ravel_multi_index(winner_coordinates, som_shape)

print(len(np.unique(cluster_index)))

# A clustering result satisfies homogeneity if all of its clusters contain only data points which are members of a single class.
print(homogeneity_score(labels_true_1, cluster_index))
print(homogeneity_score(labels_true_2, cluster_index))
print(completeness_score(labels_true_1, cluster_index))
print(completeness_score(labels_true_2, cluster_index))

 [ 500 / 500 ] 100% - 0:00:00 left 
 quantization error: 0.2632484137640619
5
0.12131178703517663
0.23088096213841064
0.4607989484400614
0.37766716300391173


# Hyperparameter tuning

In [11]:
def train_som(x, y, input_len, sigma, learning_rate, iterations): 
    som = MiniSom(x=x, 
                  y=y, 
                  input_len = input_len, 
                  sigma = sigma, 
                  learning_rate = learning_rate)
    som.random_weights_init(marker_data)
    # training
    start_time = time.time()
    som.train_random(marker_data, iterations)
    elapsed_time = time.time() - start_time
    print(elapsed_time, " seconds")
    return som

In [12]:
# set hyperparameters
rows_data = marker_data.shape[0]
x = int(np.sqrt(5 * np.sqrt(rows_data)))
print("x is {}".format(x))
y = x
input_len = marker_data.shape[1]
sigma = 0.003
learning_rate = 5
iterations = 100

space = {
    'sig': hp.uniform("sig", 0.001, 5), 
    'learning_rate': hp.uniform("learning_rate", 0.001, 5),
}

def som_fn(space): 
    sig = space['sig']
    learning_rate = space['learning_rate']
    val = MiniSom(x=x, 
                  y=x, 
                  input_len=input_len,
                  sigma=sig,
                  learning_rate=learning_rate
                  ).quantization_error(marker_data)
    print(val)
    return{'loss': val, 'status': STATUS_OK}

trials = Trials()
best = fmin(fn = som_fn, 
            space=space, 
            algo = tpe.suggest, 
            max_evals=15, 
            trials=trials)

print('best: {}'.format(best))

for i, trial in enumerate(trials.trials[:2]): 
    print(i, trials)

x is 41
8.86020294720663                                      
8.9822229798672                                                               
8.897362369950365                                                             
8.947244512913256                                                             
8.89077440911263                                                              
8.864929767196456                                                             
8.885122775151627                                                             
8.848506465478753                                                             
8.900297765448503                                                              
8.844399738149017                                                              
8.946221087161417                                                               
8.875424649474915                                                               
8.90182505202143                                              

In [13]:
sigma = best['sig']
learning_rate = best['learning_rate']
print(x, y, sigma, learning_rate)

41 41 1.521233099971735 4.36919546568751


In [14]:
som = train_som(x, y, input_len, sigma, learning_rate, iterations=500)

# each neuron represents a cluster
winner_coordinates = np.array([som.winner(x) for x in marker_data]).T
# with np.ravel_multi_index we convert the bidimensional
# coordinates to a monodimensional index
som_shape = (grid_size, grid_size)
cluster_index = np.ravel_multi_index(winner_coordinates, som_shape)

print(len(np.unique(cluster_index)))

# A clustering result satisfies homogeneity if all of its clusters contain only data points which are members of a single class.
print(homogeneity_score(labels_true_1, cluster_index))
print(homogeneity_score(labels_true_2, cluster_index))
print(completeness_score(labels_true_1, cluster_index))
print(completeness_score(labels_true_2, cluster_index))

0.07358479499816895  seconds
1421
0.464305687282738
0.5701669235664616
0.1957020433679857
0.10349181408968848
